In [53]:
# Loading required libraries
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import glob
import os
%pylab inline
matplotlib.rcParams.update({'font.size': 16})

Populating the interactive namespace from numpy and matplotlib


In [54]:
#input_file_path = '/home/cusp/kvp246/capstone/spatial/tlc/unmetdemand/'
#files = glob.glob(os.path.join(input_file_path, "*.csv"))
#files

In [55]:
bbl_unmet_metrics = pd.read_csv("wav_overall_unmet_ratio.csv",header=None)
#bbl_unmet_metrics = pd.read_csv(input_file, header=None)

#bbl_number, total_demand, total_freemins, overall unmet ratio
bbl_unmet_metrics.rename(columns={0:"bbl_number", 1:"total_demand", 2:"total_freemins",3:"total_unmet_ratio"}, inplace=True)
print bbl_unmet_metrics.shape
print bbl_unmet_metrics.info()
bbl_unmet_metrics.head()

(166850, 4)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 166850 entries, 0 to 166849
Data columns (total 4 columns):
bbl_number           166850 non-null int64
total_demand         166850 non-null int64
total_freemins       166850 non-null int64
total_unmet_ratio    166850 non-null float64
dtypes: float64(1), int64(3)
memory usage: 6.4 MB
None


,bbl_number,total_demand,total_freemins,total_unmet_ratio
0,1015450052,93,523,0.177820
1,1007237501,640,5448,0.117474
2,1016800048,1,28,0.035714
3,1013850032,658,6825,0.096410
4,1011970027,9,157,0.057325


In [56]:
bbl_unmet_metrics.bbl_number.count()

166850

In [57]:
bbl_unmet_metrics.total_demand.describe()

count    166850.000000
mean         58.422997
std         846.288241
min           1.000000
25%           1.000000
50%           3.000000
75%          12.000000
max      224013.000000
Name: total_demand, dtype: float64

In [58]:
bbl_unmet_metrics.total_freemins.describe()

count     166850.000000
mean        1025.918771
std        22801.152097
min            1.000000
25%           37.000000
50%          136.000000
75%          551.000000
max      8206852.000000
Name: total_freemins, dtype: float64

In [59]:
bbl_unmet_metrics.total_unmet_ratio.describe()

count    166850.000000
mean          0.072372
std           0.114711
min           0.000072
25%           0.013189
50%           0.038462
75%           0.090909
max           3.200000
Name: total_unmet_ratio, dtype: float64

In [60]:
allbbl_field = pd.read_csv('All_bbl.csv')

In [61]:
allbbl_field.columns

Index([u'Unnamed: 0', u'BBL', u'geometry', u'Borough', u'LandUse'], dtype='object')

In [62]:
allbbl_field = allbbl_field.drop(['Unnamed: 0'],axis=1).rename(columns={'BBL':'bbl_number'})

In [63]:
bbl_meaning = pd.read_excel('BBL_code_meaning.xlsx')

In [64]:
bbl_meaning = bbl_meaning.rename(columns={'Code':'LandUse'})

In [65]:
allbbl_field_final = pd.merge(allbbl_field,bbl_meaning,on='LandUse',how='left')

In [66]:
allbbl_field_final.head(2)

,bbl_number,geometry,Borough,LandUse,Desc
0,1015450052,"POLYGON ((-73.95296500000001 40.775501, -73.95...",MN,4,Mixed Residential & Commercial Buildings
1,1007237501,"POLYGON ((-74.000308 40.748145, -74.000641 40....",MN,3,Multi-Family Elevator Buildings


In [67]:
allbbl_field_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 857216 entries, 0 to 857215
Data columns (total 5 columns):
bbl_number    857216 non-null int64
geometry      857216 non-null object
Borough       857216 non-null object
LandUse       854147 non-null float64
Desc          854147 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 39.2+ MB


# determining the cut off for demand

In [68]:
global perc_33th_value, perc_66th_value
## unmet
def bbl_bucketing_unmet(row):
    
    # BBL bucketing
    if (row['total_demand']!=0) & (row['total_freemins']!=0) & (row['total_unmet_ratio']>=perc_66th_value):
        return "Top33_P"
    elif (row['total_demand']!=0) & (row['total_freemins']!=0) & (row['total_unmet_ratio']>=perc_33th_value) & (row['total_unmet_ratio']<perc_66th_value):
        return "Middle33_P"
    else:
        return "Bottom33_P"
    

In [69]:
## for wav total demand

def bbl_bucketing_total_demand(row):
    
    # BBL bucketing
    if (row['total_demand']!=0) & (row['total_freemins']!=0) & (row['total_demand']>=perc_66th_value):
        return "Top33_P"
    elif (row['total_demand']!=0) & (row['total_freemins']!=0) & (row['total_demand']>=perc_33th_value) & (row['total_demand']<perc_66th_value):
        return "Middle33_P"
    else:
        return "Bottom33_P"

In [70]:
## for wav total demand

def bbl_bucketing_total_free(row):
    
    # BBL bucketing
    if (row['total_demand']!=0) & (row['total_freemins']!=0) & (row['total_freemins']>=perc_66th_value):
        return "Top33_P"
    elif (row['total_demand']!=0) & (row['total_freemins']!=0) & (row['total_freemins']>=perc_33th_value) & (row['total_freemins']<perc_66th_value):
        return "Middle33_P"
    else:
        return "Bottom33_P"

In [71]:
## overall wav consideration 

In [72]:
## for unmet
data_u = bbl_unmet_metrics
perc_33th_value = np.percentile(data_u['total_unmet_ratio'], 33)
perc_66th_value = np.percentile(data_u['total_unmet_ratio'], 66)
print perc_33th_value, perc_66th_value
data_u['spatial_bkt'] = data_u.apply(bbl_bucketing_unmet, axis=1)
print data_u['spatial_bkt'].value_counts()
#data[['bbl_number', 'spatial_bkt']].to_csv("overall_bbl_counts.csv", index=None)

0.0192307692308 0.0666666666667
Top33_P       57146
Middle33_P    55011
Bottom33_P    54693
dtype: int64


In [73]:
## for totaldemand
data_t = bbl_unmet_metrics
perc_33th_value = np.percentile(data_t['total_demand'], 33)
perc_66th_value = np.percentile(data_t['total_demand'], 66)
print perc_33th_value, perc_66th_value
data_t['spatial_bkt'] = data_t.apply(bbl_bucketing_total_demand, axis=1)
print data_t['spatial_bkt'].value_counts()

2.0 6.0
Top33_P       60628
Bottom33_P    54656
Middle33_P    51566
dtype: int64


In [74]:
## for free minute :
data_f = bbl_unmet_metrics
perc_33th_value = np.percentile(data_f['total_freemins'], 33)
perc_66th_value = np.percentile(data_f['total_freemins'], 66)
print perc_33th_value, perc_66th_value
data_f['spatial_bkt'] = data_f.apply(bbl_bucketing_total_free, axis=1)
print data_f['spatial_bkt'].value_counts()

57.0 321.0
Top33_P       56770
Middle33_P    55410
Bottom33_P    54670
dtype: int64


In [75]:
dat_u = pd.merge(data_u,allbbl_field_final,on='bbl_number',how='inner')

In [76]:
dat_u[['bbl_number','spatial_bkt','geometry']].to_csv('wav_bucket_unmet.csv')

In [77]:
dat_t = pd.merge(data_t,allbbl_field_final,on='bbl_number',how='inner')

In [78]:
dat_t[['bbl_number','spatial_bkt','geometry']].to_csv('wav_bucket_total.csv')

In [79]:
dat_f = pd.merge(data_f,allbbl_field_final,on='bbl_number',how='inner')

In [80]:
dat_f[['bbl_number','spatial_bkt','geometry']].to_csv('wav_bucket_free.csv')

In [ ]:
data_1[data_1.total_demand>100].sort('total_unmet_ratio', ascending=False)[:20]

### Overall Segment

In [51]:
data = bbl_unmet_metrics_1.groupby('bbl_number')[['total_demand', 'total_freemins']].sum()
data = data.reset_index()
data['total_unmet_ratio'] = data['total_demand']*1.0/data['total_freemins']
data_1 = data[(data.total_demand!=0) & (data.total_freemins!=0)]
perc_33th_value = np.percentile(data_1['total_unmet_ratio'], 33)
perc_66th_value = np.percentile(data_1['total_unmet_ratio'], 66)
print perc_33th_value, perc_66th_value
data['spatial_bkt'] = data.apply(bbl_bucketing, axis=1)
print data['spatial_bkt'].value_counts()
#data[['bbl_number', 'spatial_bkt']].to_csv("overall_bbl_counts.csv", index=None)

0.00854700854701 0.032967032967
No Demand      472728
Top33_P        105125
Middle33_P     102057
Bottom33_P     101855
No Coverage     75304
No Supply         147
dtype: int64


In [52]:
data.head(2)

,bbl_number,total_demand,total_freemins,total_unmet_ratio,spatial_bkt
0,1000010010,97,1283,0.075604,Top33_P
1,1000010101,4,24,0.166667,Top33_P


In [53]:
dat = pd.merge(data,allbbl_field_final,on='bbl_number',how='inner')

In [56]:
dat[dat.Borough=='MN'].to_csv('Overallbucketing_manhaten.csv')

In [ ]:
#dat[['bbl_number','spatial_bkt','geometry']].to_csv('weekend[8-12pm].csv')

In [34]:
dat[dat.total_demand>233].sort('total_unmet_ratio', ascending=False)[:50][['bbl_number','geometry','spatial_bkt']].to_csv('test.csv')

In [36]:
dat[dat.total_demand>233].sort('total_unmet_ratio', ascending=False)[:50]

,bbl_number,total_demand,total_freemins,total_unmet_ratio,spatial_bkt,geometry,Borough,LandUse,Desc
497776,4035390100,374,221,1.692308,Top33_P,"POLYGON ((-73.904855 40.694955, -73.905643 40....",QN,6,Industrial & Manufacturing
6570,1004950001,22182,37748,0.587634,Top33_P,"POLYGON ((-73.996787 40.722551, -73.9968960000...",MN,4,Mixed Residential & Commercial Buildings
9245,1006100017,2414,4539,0.531835,Top33_P,"POLYGON ((-74.00173700000001 40.7343, -74.0017...",MN,10,Parking Facilities
4742,1004290002,11284,21360,0.528277,Top33_P,"POLYGON ((-73.987927 40.723327, -73.9879860000...",MN,4,Mixed Residential & Commercial Buildings
494570,4034440041,1489,2920,0.509932,Top33_P,"POLYGON ((-73.911412 40.700081, -73.9115769999...",QN,4,Mixed Residential & Commercial Buildings
8727,1005890041,5090,10469,0.486197,Top33_P,"POLYGON ((-74.00183800000001 40.730752, -74.00...",MN,5,Commercial & Office Buildings
11230,1007400010,914,1882,0.485654,Top33_P,"POLYGON ((-74.003235 40.742169, -74.003399 40....",MN,2,Multi-Family Walk-Up Buildings
4743,1004290004,5401,11171,0.483484,Top33_P,"POLYGON ((-73.987791 40.723343, -73.987832 40....",MN,4,Mixed Residential & Commercial Buildings
141408,3005310003,281,598,0.469900,Top33_P,"POLYGON ((-74.010564 40.67891, -74.010609 40.6...",BK,4,Mixed Residential & Commercial Buildings
42697,2000008900,18773,40235,0.466584,Top33_P,"MULTIPOLYGON (((-73.90579200000001 40.899678, ...",BX,NaN,NaN


In [ ]:
dat.head(2)

In [ ]:
dat.LandUse.hist()

In [ ]:
dat[['Desc']].to_csv('Landuse.csv')